# Agrupamento Fuzzy

### Grupo
- Gabriel Camatta Zanotelli - 2018020140
- Nander Santos do Carmo - 2018019931

### Bibliotecas e dados

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat
import math
import skfuzzy
import pandas as pd
import random as rand
import torch
from torch.utils.data import TensorDataset, DataLoader
from scipy.io import loadmat
import sys
sys.path.append("./anfis-pytorch/")
from anfispytorch.experimental import train_anfis, test_anfis, plot_all_mfs
from anfispytorch.membership import GaussMembFunc, make_gauss_mfs
import anfispytorch.anfis1 as ANFIS

iris = open('Iris.csv', mode='r')
breast = open('Breast.csv', mode='r')

---
## 2- Aproximação de Funções

Utilizaremos um sistema ANFIS de tipo Tageki-Sugueno de Ordem 1 de forma a aproximar uma função na forma sen(x) definida no espaço [0,2] utilizando uma N=80 pontos utilizando a seguinte equação:

![Formula senóide com ruído](./senoide.png 'Senóide com ruído')

### Funções de Pertinência

Por experiência temos que, para representar uma senóide de forma precisa, devemos utilizar três regras de pertinência.

In [ ]:
# Espaço definido
N = 80
x = np.linspace(0, 2*math.pi, N)

# valores de ativação de cada regra
fun_pertinencia1 = skfuzzy.gaussmf(x, 0, 1.25)
fun_pertinencia2 = skfuzzy.gaussmf(x, math.pi, 1.25)
fun_pertinencia3 = skfuzzy.gaussmf(x, 2*math.pi, 1.25)

# plota as funções de pertinência
plt.plot(x, fun_pertinencia1, x, fun_pertinencia2, x, fun_pertinencia2, x, fun_pertinencia3)
plt.title("Funções de pertinência da senóide com 3 regras")
plt.xlabel("Radianos")
plt.ylabel("Pertinência")
plt.show()

### Geração do conjunto de dados

Gera o conjunto de dados aleatórios da senóide e separa conjunto de treino e teste.

In [ ]:
def gera_dados_treino(size):
    perc_train = 0.75
    x_space = np.linspace(0, 2*math.pi, size)

    # Conjunto de dados de trainamento e teste
    x_rand_id = np.linspace(0, size-1, size).astype(int)
    rand.shuffle(x_rand_id)

    x_train = x_space[x_rand_id[0:int(size*perc_train)]]
    x_te = x_space[x_rand_id[int(size*perc_train):size]]

    y = np.sin(x_space) + 0.1*np.random.randn(size)
    y_train = y[x_rand_id[0:int(size*perc_train)]]
    y_te = y[x_rand_id[int(size*perc_train):size]]

    # Plota dados de treino (azul) e de teste (laranja)
    plt.scatter(x_train,y_train)
    plt.scatter(x_te,y_te)

    return [x_train, x_te, y_train, y_te]


def gera_dados_modelo(x_train, y_train):
    x_train_aux = torch.tensor([[val] for val in x_train], dtype=torch.float)
    y_train_aux = torch.tensor([[val] for val in y_train], dtype=torch.float)
    return DataLoader(TensorDataset(x_train_aux, y_train_aux))

### Cria modelo ANFIS

In [ ]:
def instantiate_problem():
    x1 = [('x1', make_gauss_mfs(math.pi/2, [0.0, math.pi, 2*math.pi]))]
    x2 = ['x2']
    return ANFIS.AnfisNet('Senoide', x1, x2)

### Executa modelo

In [ ]:
anfis_model = instantiate_problem()
dados_treino = gera_dados_treino(N)


dados_treino_modelados = gera_dados_modelo(dados_treino[0], dados_treino[2])

train_anfis(anfis_model, dados_treino_modelados, N, True)


---
## 3- Previsão de uma Séria Temporal

A série temporal de *Mackey-Glass* é definida pela seguinte formula:

![Série de Mackey-Glass](./serie_temporal.png 'Formato da série temporal')

In [ ]:
import scipy.io
dados = scipy.io.loadmat('mg.mat')
mat = []
for i in dados['x']:
    mat.append(i[0])

In [2]:
# loading data set
data = loadmat('mg.mat')
data = pd.DataFrame(data['x'])
sgn = data.values.reshape(-1,)
len(sgn)

#train test
size = len(sgn)
train_s = int(size * 0.8)
test_s = size - train_s
x_train = sgn[:train_s]
x_test = sgn[train_s:]

def model_serie():
    invardefs = [
            ('xm18', make_gauss_mfs(0.1, [0.425606, 1.313696])),
            ('xm12', make_gauss_mfs(0.1, [0.425606, 1.313696])),
            ('xm6',  make_gauss_mfs(0.1, [0.425606, 1.313696])),
            ('x',    make_gauss_mfs(0.1, [0.425606, 1.313696])),
            ]
    outvars = ['xp6']
    model = ANFIS.AnfisNet('model_define', invardefs, outvars)
    return model


def our_data(data):
    num_cases = len(data) - 18
    x = torch.zeros((num_cases, 4))
    y = torch.zeros((num_cases, 1))
    for t in range(18, len(data)-6):
            values = [data[t-18],data[t-12],data[t-6],data[t],data[t+6]]
            x[t-18] = torch.tensor(values[0:4])
            y[t-18] = values[4]
    dl = DataLoader(TensorDataset(x, y), batch_size=1024, shuffle=True)
    return dl


model = model_serie()
train_data = our_data(x_train)
train_anfis(model, train_data, 100, True)
test_data = our_data(x_test)
test_anfis(model, test_data, True)

### Training for 100 epochs, training size = 782 cases


ValueError: too many values to unpack (expected 2)

---
## 4- Classificação de Padrões